# Segmentation Algorithm Workflows

This notebook demonstrates IceFloeTracker.jl's segmentation algorithms 
with very basic examples of their use.

In [ ]:
# Setup environment
using Pkg
Pkg.add("IceFloeTracker")
Pkg.add("Images")

     Cloning git-repo `/workspaces/IceFloeTracker.jl`
    Updating git-repo `/workspaces/IceFloeTracker.jl`
   Resolving package versions...
   Installed Xorg_xkbcomp_jll ───────────── v1.4.7+0
   Installed libdecor_jll ───────────────── v0.2.2+0
   Installed libfdk_aac_jll ─────────────── v2.0.4+0
   Installed Opus_jll ───────────────────── v1.5.2+0
   Installed x265_jll ───────────────────── v4.1.0+0
   Installed Libmount_jll ───────────────── v2.41.1+0
   Installed RelocatableFolders ─────────── v1.0.1
   Installed GR_jll ─────────────────────── v0.73.17+0
   Installed Measures ───────────────────── v0.3.2
   Installed Xorg_xcb_util_image_jll ────── v0.4.1+0
   Installed Grisu ──────────────────────── v1.0.2
   Installed Xorg_xcb_util_wm_jll ───────── v0.4.2+0
   Installed Unitful ────────────────────── v1.24.0
   Installed Contour ────────────────────── v0.6.3
   Installed PlotUtils ──────────────────── v1.4.3
   Installed RecipesPipeline ────────────── v0.6.12
   Installed Xorg_xc

In [ ]:
# Load packages
using IceFloeTracker: LopezAcosta2019, LopezAcosta2019Tiling, Watkins2025GitHub
using Images: segment_mean, labels_map, SegmentedImage, RGB

## Load the images

Load the dataset from https://github.com/danielmwatkins/ice_floe_validation_dataset using the `Watkins2025GitHub` data loader.

In [ ]:
data_loader = Watkins2025GitHub(; ref="a451cd5e62a10309a9640fbbe6b32a236fcebc70")

The available data are listed in the metadata field:

In [ ]:
data_loader().metadata

For the example, we choose a single case from Baffin Bay in May 2022.

In [ ]:
dataset = data_loader(c-> c.case_number == 6 && c.satellite == "terra")
case = first(dataset)

The data include the true-color image:

In [ ]:
truecolor = RGB.(case.modis_truecolor) # TODO: remove RGB cast

... a false-color image:

In [ ]:
falsecolor = RGB.(case.modis_falsecolor) # TODO: remove RGB cast

... and a landmask, which in this particular case is empty:

In [ ]:
landmask = RGB.(case.modis_landmask) # TODO: remove RGB cast

## Run the segmentation algorithm

The segmentation algorithm is an object with parameters as follows:

In [ ]:
segmentation_algorithm = LopezAcosta2019()

If we wanted to modify the options, we could include those in the call above. 
See the documentation for `LopezAcosta2019` for details.
The default parameters are as follows:

In [ ]:
dump(segmentation_algorithm)

Run the algorithm as follows:

In [ ]:
segments = segmentation_algorithm(truecolor, falsecolor, landmask)

To show the results with each segment marked using its mean color:

In [ ]:
map(i -> segment_mean(segments, i), labels_map(segments))

We can do the same with the falsecolor image:

In [ ]:
# Get the labels_map
segments_falsecolor = SegmentedImage(falsecolor, labels_map(segments))
map(i -> segment_mean(segments_falsecolor, i), labels_map(segments_falsecolor))

## Run the segmentation algorithm with tiling
The "tiling" version of the algorithm is an object:

In [ ]:
segmentation_algorithm_with_tiling = LopezAcosta2019Tiling()

It has more configurable parameters. 
For details, see the documentation of LopezAcosta2019Tiling.
The default parameters are as follows:

In [ ]:
dump(segmentation_algorithm_with_tiling)

In [ ]:
display("text/markdown", @doc LopezAcosta2019Tiling)

In [ ]:
segments = segmentation_algorithm_with_tiling(truecolor, falsecolor, landmask)

To show the results with each segment marked using its mean color:

In [ ]:
map(i -> segment_mean(segments, i), labels_map(segments))

With the falsecolor image:

In [ ]:
# Get the labels_map
segments_falsecolor = SegmentedImage(falsecolor, labels_map(segments))
map(i -> segment_mean(segments_falsecolor, i), labels_map(segments_falsecolor))